In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from cftime import num2date
import datetime as dt

In [2]:
dirnc_3h = '/Volumes/L4/ERAinterim/nc/'
dirnc_daily = '/Volumes/L4/ERAinterim/nc_daily/'

In [3]:
def make_daily_average(ds):
    """ compute daily average from 3 hours cumulative"""
    grp = ds.groupby(ds.time.dt.hour)
    midnight = grp.groups[0]
    noon = grp.groups[12]
    ds.load()
    daily_avg = xr.Dataset()
    # geo coordinates
    daily_avg['lon'] = ds['lon']
    daily_avg['lat'] = ds['lat']
    # daily precip
    rho_w = 1000  # kg.m-3
    seconds_per_day = 86400 # s.day-1
    # kg.m-2.s-1 = kg.m-3 * m.day-1 / s.day-1
    precip = rho_w * (ds.TP.isel(time=midnight).values + ds.TP.isel(time=noon)).values / seconds_per_day
    daily_avg['precip'] = xr.DataArray(data=precip, dims=['time', 'lat', 'lon'])
    # new time axis
    year = ds.time[0].dt.year.values
    ndays = len(ds.time) / 8.
    dateref = dt.datetime(1900,1,1,0,0,0)
    datestart = dt.datetime(year,1,1,12,0,0)
    time = (datestart - dateref).total_seconds() + seconds_per_day * np.arange(ndays)
    daily_avg['time'] = time
    daily_avg['time'].attrs['units'] = "seconds since 1900-01-01 0:00:00"
    daily_avg['time'].attrs['calendar'] = "gregorian"
    
    return daily_avg

In [4]:
for year in np.arange(1979,2018+1):
    print(f'working on year {year}')
    ds = xr.open_dataset(f'{dirnc_3h}/precip_ERAinterim_{year}.nc')
    ds_daily = make_daily_average(ds)
    ds_daily.to_netcdf(f'{dirnc_daily}/precip_ERAinterim_{year}_daily.nc')

working on year 1979
working on year 1980
working on year 1981
working on year 1982
working on year 1983
working on year 1984
working on year 1985
working on year 1986
working on year 1987
working on year 1988
working on year 1989
working on year 1990
working on year 1991
working on year 1992
working on year 1993
working on year 1994
working on year 1995
working on year 1996
working on year 1997
working on year 1998
working on year 1999
working on year 2000
working on year 2001
working on year 2002
working on year 2003
working on year 2004
working on year 2005
working on year 2006
working on year 2007
working on year 2008
working on year 2009
working on year 2010
working on year 2011
working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017
working on year 2018


In [5]:
ds_daily

<xarray.Dataset>
Dimensions:  (lat: 256, lon: 512, time: 365)
Coordinates:
  * lat      (lat) float64 89.46 88.77 88.07 87.37 ... -88.07 -88.77 -89.46
  * lon      (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
  * time     (time) float64 3.724e+09 3.724e+09 ... 3.755e+09 3.755e+09
Data variables:
    precip   (time, lat, lon) float64 5.298e-06 5.314e-06 ... 3.091e-07